This exercise will require you to pull some data from https://data.nasdaq.com/ (formerly Quandl API).


As a first step, you will need to register a free account on the https://data.nasdaq.com/ website.


After you register, you will be provided with a unique API key, that you should store:

_Note_: Use a `.env` file and put your key in there and `python-dotenv` to access it in this notebook.

The code below uses a key that was used when generating this project but has since been deleted. Never submit your keys to source control. There is a `.env-example` file in this repository to illusrtate what you need. Copy that to a file called `.env` and use your own api key in that `.env` file. Make sure you also have a `.gitignore` file with a line for `.env` added to it.

The standard Python gitignore is [here](https://github.com/github/gitignore/blob/master/Python.gitignore) you can just copy that.


In [1]:
# get api key from your .env file
import os
from dotenv import load_dotenv

load_dotenv()
API_KEY = os.getenv("NASDAQ_API_KEY")

# print(API_KEY)

Nasdaq Data has a large number of data sources, but, unfortunately, most of them require a Premium subscription. Still, there are also a good number of free datasets.


For this mini project, we will focus on equities data from the Frankfurt Stock Exhange (FSE), which is available for free. We'll try and analyze the stock prices of a company called Carl Zeiss Meditec, which manufactures tools for eye examinations, as well as medical lasers for laser eye surgery: https://www.zeiss.com/meditec/int/home.html. The company is listed under the stock ticker AFX_X.


You can find the detailed Nasdaq Data API instructions here: https://docs.data.nasdaq.com/docs/in-depth-usage


While there is a dedicated Python package for connecting to the Nasdaq API, we would prefer that you use the _requests_ package, which can be easily downloaded using _pip_ or _conda_. You can find the documentation for the package here: http://docs.python-requests.org/en/master/


Finally, apart from the _requests_ package, you are encouraged to not use any third party Python packages, such as _pandas_, and instead focus on what's available in the Python Standard Library (the _collections_ module might come in handy: https://pymotw.com/3/collections/).
Also, since you won't have access to DataFrames, you are encouraged to us Python's native data structures - preferably dictionaries, though some questions can also be answered using lists.
You can read more on these data structures here: https://docs.python.org/3/tutorial/datastructures.html


Keep in mind that the JSON responses you will be getting from the API map almost one-to-one to Python's dictionaries. Unfortunately, they can be very nested, so make sure you read up on indexing dictionaries in the documentation provided above.


In [2]:
# First, import the relevant modules
import requests
import json
import statistics as stats

url = f"https://data.nasdaq.com/api/v3/datasets/FSE/AFX_X/data.json"


api_param = {"api_key": API_KEY}

req = requests.get(url=url, params=api_param)
data = req.json()


Note: API's can change a bit with each version, for this exercise it is reccomended to use the nasdaq api at `https://data.nasdaq.com/api/v3/`. This is the same api as what used to be quandl so `https://www.quandl.com/api/v3/` should work too.

Hint: We are looking for the `AFX_X` data on the `datasets/FSE/` dataset.


In [3]:
# Now, call the Nasdaq API and pull out a small sample of the data (only one day) to get a glimpse
# into the JSON structure that will be returned


url = f"https://data.nasdaq.com/api/v3/datasets/FSE/AFX_X/data.json"
one_row_params = {"rows": 1, "api_key": API_KEY}

req = requests.get(url=url, params=one_row_params)
data = req.json()

In [4]:
# Inspect the JSON structure of the object you created, and take note of how nested it is,
# as well as the overall structure
data


{'dataset_data': {'limit': 1,
  'transform': None,
  'column_index': None,
  'column_names': ['Date',
   'Open',
   'High',
   'Low',
   'Close',
   'Change',
   'Traded Volume',
   'Turnover',
   'Last Price of the Day',
   'Daily Traded Units',
   'Daily Turnover'],
  'start_date': '2000-06-07',
  'end_date': '2020-12-01',
  'frequency': 'daily',
  'data': [['2020-12-01',
    112.2,
    112.2,
    111.5,
    112.0,
    None,
    51.0,
    5703.0,
    None,
    None,
    None]],
  'collapse': None,
  'order': None}}

These are your tasks for this mini project:

1. Collect data from the Franfurt Stock Exchange, for the ticker AFX_X, for the whole year 2017 (keep in mind that the date format is YYYY-MM-DD).
2. Convert the returned JSON object into a Python dictionary.
3. Calculate what the highest and lowest opening prices were for the stock in this period.
4. What was the largest change in any one day (based on High and Low price)?
5. What was the largest change between any two days (based on Closing Price)?
6. What was the average daily trading volume during this year?
7. (Optional) What was the median trading volume during this year. (Note: you may need to implement your own function for calculating the median.)


1. Collect data from the Franfurt Stock Exchange, for the ticker AFX_X, for the whole year 2017 (keep in mind that the date format is YYYY-MM-DD).
2. Convert the returned JSON object into a Python dictionary.


In [5]:
url = f"https://data.nasdaq.com/api/v3/datasets/FSE/AFX_X/data.json"
one_year_params = {
    "start_date": "2017-01-01",
    "end_date": "2017-12-31",
    "limit": None,
    "api_key": API_KEY,
}

req = requests.get(url, params=one_year_params)
data = req.json()
for k, v in data["dataset_data"].items():
    if k != "data":
        print(f"{k}: {v}")

limit: None
transform: None
column_index: None
column_names: ['Date', 'Open', 'High', 'Low', 'Close', 'Change', 'Traded Volume', 'Turnover', 'Last Price of the Day', 'Daily Traded Units', 'Daily Turnover']
start_date: 2017-01-01
end_date: 2017-12-31
frequency: daily
collapse: None
order: None


3. Calculate what the highest and lowest opening prices were for the stock in this period.


In [6]:
nested = data["dataset_data"]
headers = nested["column_names"]

afx_x_2017 = {}


afx_x_2017["open"] = [float(row[1]) if row[1] else -1.0 for row in nested["data"]]

lowest_open_value = sorted(filter(lambda x: x > 0, afx_x_2017["open"]))[0]
highest_open_value = sorted(afx_x_2017["open"])[-1]
print(
    f"Lowest open value: {lowest_open_value:.2f}\nHighest open value: {highest_open_value:.2f}"
)

Lowest open value: 34.00
Highest open value: 53.11


4. What was the largest change in any one day (based on High and Low price)?


In [7]:
afx_x_2017["low"] = [float(row[3]) for row in nested["data"]]
afx_x_2017["high"] = [float(row[2]) for row in nested["data"]]

high_low_diff = [
    abs(high - low) for (high, low) in zip(afx_x_2017["high"], afx_x_2017["low"])
]

# largest change in any oneday
largest_change_hl_1d = sorted(high_low_diff)[-1]
print(
    f"Largest change in any one-day based on High and Low prices: {largest_change_hl_1d:.2f}"
)

Largest change in any one-day based on High and Low prices: 2.81


5. What was the largest change between any two days (based on Closing Price)?


In [8]:
afx_x_2017["close"] = [float(row[4]) for row in nested["data"]]
close_values_diff = [
    abs(afx_x_2017["close"][i] - afx_x_2017["close"][i - 1])
    for i in range(1, len(afx_x_2017["close"]))
]
max_change_close = max(close_values_diff)
print(
    f"Largest change between any 2 days based on Closing Price: {max_change_close:.2f}"
)

Largest change between any 2 days based on Closing Price: 2.56


6. What was the average daily trading volume during this year?
7. (Optional) What was the median trading volume during this year. (Note: you may need to implement your own function for calculating the median.)


In [9]:
afx_x_2017["traded_volume"] = [int(row[6]) for row in nested["data"]]
avg_trading_volume = stats.mean(afx_x_2017["traded_volume"])
med_trading_volume = stats.median(afx_x_2017["traded_volume"])
print(f"Average daily trading volume during this year: {avg_trading_volume:.0f} ")
print(f"Median daily trading volume during this year: {med_trading_volume:.0f} ")

Average daily trading volume during this year: 89124 
Median daily trading volume during this year: 76286 


In [10]:
def get_median(a_lst):
    lst = a_lst[:]
    n = len(lst)
    lst.sort()
    if n % 2:
        return lst[(n - 1) // 2]
    else:
        return (lst[n // 2] + lst[n // 2 - 1]) / 2


In [11]:
my_med_trading_volume = get_median(afx_x_2017["traded_volume"])

print(f"Median daily trading volume during this year: {med_trading_volume:.0f} ")

Median daily trading volume during this year: 76286 
